# Building symbolic metamodels

A symbolic metamodel takes as an input a machine learning model, and outputs a symbolic equation describing its response surface as illustrated in the Figure below. This notebook provides the steps needed for building a symbolic metamodel for an XGBoost model fitted to the "UCI absenteeism at work dataset: https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work

<img src="images/FigB1.png", width=640>

We first import the necessary modules for symbolic metamodeling

In [1]:
from pysymbolic.algorithms.symbolic_metamodeling import *

Next, we import the necessary libraries fordata processing, splitting the data into training and testing samples, and XGBoost model fitting 

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


data          = pd.read_csv("data/absenteeism.csv", delimiter=';')

feature_names = ['Transportation expense', 'Distance from Residence to Work',
                 'Service time', 'Age', 'Work load Average/day ', 'Hit target',
                 'Disciplinary failure', 'Education', 'Son', 'Social drinker',
                 'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index']

scaler        = MinMaxScaler(feature_range=(0, 1))
X             = scaler.fit_transform(data[feature_names])
Y             = ((data['Absenteeism time in hours'] > 4) * 1) 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

model         = XGBClassifier()

model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Let us examine the AUC-ROC performance of the fitted XGBoost model on the test data

In [4]:
roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1])

np.float64(0.6981398378691372)

How much better does XGBoost perform compared to a standard logistic regression?

In [5]:
model_L = LogisticRegression()

model_L.fit(X_train, Y_train)

roc_auc_score(Y_test, model_L.predict_proba(X_test)[:, 1])

np.float64(0.6709973943254198)

In [6]:
metamodel = symbolic_metamodel(model, X_train)

metamodel.fit(num_iter=10, batch_size=X_train.shape[0], learning_rate=.01)

Tuning basis functions in parallel...
Optimizing metamodel...
Iteration 0, Loss: 4.3192, LR: 0.010000
Iteration 5, Loss: 4.3169, LR: 0.010000


In [8]:
Y_metamodel = metamodel.evaluate(X_test)

roc_auc_score(Y_test, Y_metamodel)

np.float64(0.740047770700637)

In [9]:
# Get symbolic expressions
exact_expr, approx_expr = metamodel.symbolic_expression()

# Print the expressions
print("Exact expression:")
print(exact_expr)
print("\nApproximate expression:")
print(approx_expr)



AttributeError: 'Mul' object has no attribute 'expression'

In [9]:
# Calculate gradients for specific instances
from sympy import Symbol, exp, symbols, sympify, re, im  # Added re import from sympy
X_sample = X[:5]  # Get first 5 instances
gradients = metamodel.get_instancewise_scores(X_sample)

NameError: name 're' is not defined

Now let us see how this metamodel performs using the **evaluate** method...

It performs close to the original XGBoost model! Now let us see the exact symbolic equation of the model in terms of Meijer-G functions

By invoking the **.fit()** method in the **symbolic_metamodel** class, we essentially transform the XGBoost model to a space of interpretable symbolic equations as shown in the Figure below, without much loss in predictive accuracy. 

<img src="images/FigB2.png", width=320>

Now we show how to extract the exact and approximate equation $g(x)$ from the metamodel class...

In [10]:
metamodel.exact_expression

AttributeError: 'FastSymbolicMetamodel' object has no attribute 'exact_expression'

Because this equation involves Hypergeometric functions, we might prefer to work with the polynomial approximation...

In [ ]:
metamodel.approx_expression